[View in Colaboratory](https://colab.research.google.com/github/jimmyjamesarnold/Compound_Interest/blob/master/Compound_Interest.ipynb)

In [8]:
import warnings
warnings.filterwarnings('ignore')

# load pydrive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

2018-08-24 05:01:50,230 WARNING No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


In [9]:
# performance tracking
from __future__ import print_function
from pprint import pprint
from time import time
import logging

# Load packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# text and feature manipulation
import re
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Models
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
# logistic Regression
from sklearn.linear_model import LogisticRegression
# Decision Trees
from sklearn.tree import DecisionTreeClassifier
# NaiveBayes
from sklearn.naive_bayes import GaussianNB
# RandomForest
from sklearn.ensemble import RandomForestClassifier
# SVM Classifier
from sklearn.svm import SVC
# OvR Classifier
from sklearn.multiclass import OneVsRestClassifier

print(__doc__)
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

Automatically created module for IPython interactive environment


In [10]:
# load files from drive
train_import = drive.CreateFile({'id':'1KlafA7iNBDFrjFx4dA7RIEYF3_EqPEYi'})
train_import.GetContentFile('train.json')

test_import = drive.CreateFile({'id':'1SJN9ht0gaNa8OfPsOJU3PfzIFfPqKqX0'})
test_import.GetContentFile('test.json')

cmpd_import = drive.CreateFile({'id':'1Jqx15uTUd264d5T8zN11USUA2DGwoiLo'})
cmpd_import.GetContentFile('labeled_standard_contents_in_foodID_by_compoundID.csv')

2018-08-24 05:02:13,692 WARNING file_cache is unavailable when using oauth2client >= 4.0.0
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module

In [6]:
# Load foodb content and food CSVs - from other project
cmpds = pd.read_csv('labeled_standard_contents_in_foodID_by_compoundID.csv')
# melt, drop 0s, and lowercase names
cmpds = pd.melt(cmpds, id_vars=['name'], var_name='cmpd', value_name='std_content')
cmpds = cmpds[cmpds.std_content > 0]
cmpds.cmpd = cmpds.cmpd.str.replace(' ','_')
cmpds.cmpd = cmpds.cmpd.str.replace(',','_')
cmpds.name = cmpds.name.str.lower()
# construct df of measured cmpds groupby ingredient. Ignore std_content - for now.
ing_cmpd_df = cmpds.groupby(['name'])['cmpd'].apply(','.join).reset_index()

# Load train and test data (in pandas)
train = pd.read_json('train.json').set_index('id')
test = pd.read_json('test.json').set_index('id')

FileNotFoundError: ignored

In [4]:
## Load train and test data (in json)
#import json
## Dataset Preparation
#print ("Read Dataset ... ")
#def read_dataset(path):
#	return json.load(open(path))
#train = read_dataset('train.json')
#test = read_dataset('test.json')

# save indices for later
traindex = train.index
testdex = test.index
# merge sets for processing
df = pd.concat([train.drop("cuisine", axis=1), test], axis=0)
df_index = df.index

# Text Data Features
print ("Prepare text data of Train and Test ... ")
def generate_text(data):
	text_data = [" ".join(doc['ingredients']).lower() for doc in data]
	return text_data 

train_text = generate_text(train)
test_text = generate_text(test)
target = [doc['cuisine'] for doc in train]

## Label Encoding - Target 
#print ("Label Encode the Target Variable ... ")
#lb = LabelEncoder()
#y = lb.fit_transform(target)

# simple EDA
sns.countplot(y=y, order=y.value_counts().reset_index()["index"])
plt.title("Cuisine Distribution")
plt.show()

# feature engineering
print ("TF-IDF on text data ... ")
def tfidf_features(txt, flag):
    if flag == "train":
    	x = tfidf.fit_transform(txt)
    else:
	    x = tfidf.transform(txt)
    x = x.astype('float16')
    return x 

X = tfidf_features(train_text, flag="train")
X_test = tfidf_features(test_text, flag="test")

## first, use Sklearn's CountVectorizer to one hot encode.
#vect = CountVectorizer(tokenizer=lambda x: [i.strip() for i in x.split(',')])

# or, use Sklearn's TfidfVectorizer to one hot encode.
tfidf = TfidfVectorizer(tokenizer=lambda x: [i.strip() for i in x.split(',')], binary=True)

# next, define function to tokenize items in series
def inv_count(x):  # call series from pd
    dummies = tfidf.fit_transform(x.apply(','.join)) 
    df = pd.DataFrame(dummies.todense(),columns=tfidf.get_feature_names())
    return df
# Let's take an inventory of all ingredients
all_ingr = inv_count(df.ingredients)    

# get list of ingredients
pantry = all_ingr.columns.get_values()
# build dict to relate ingredients to compounds
cmpd_dict = {} 
for i in pantry:
    temp = i.replace('(','') 
    temp = temp.replace(')','')
    temp = temp.split()
    cmpd_list = []
    cmpd_list.extend([','.join(ing_cmpd_df[ing_cmpd_df.name.str.contains(j)].cmpd) for j in temp])
    cmpd_dict[i] = cmpd_list

# map recipes to cmpds using cmpd_dict
def rec_cmpdr(df): 
    recipe_dict = defaultdict(list) # dict for mapping recipe ingredients to compounds, uses defaultdict to build from empty list
    for i, row in df.iterrows(): 
        for k in row.ingredients:
            if k in cmpd_dict: # check for ingredient, if not, leave empty
                recipe_dict[i].extend(cmpd_dict[k]) # extends list of compounds for all ingredients in recipe, stored to index
            else: recipe_dict[i]=[] 
    return recipe_dict
# add compounds to df
df["compounds"] = pd.Series(rec_cmpdr(df)) 
# Now let's take an inventory of all compounds
all_cmpd = inv_count(df.compounds) 

# merge ingredient and compound data and use for training.
X = all_ingr.join(all_cmpd,lsuffix='_all_ingr', rsuffix='_all_cmpd')
# split data back into train and test
X_train = X.loc[traindex,:]
# save test case for trained model
final_test = X.loc[testdex,:]
# split training data for early training
X_TRAIN, X_test, y_train, y_test = train_test_split(X_train,y,test_size=.3,random_state=42)

FileNotFoundError: ignored

In [0]:

# Logistic Regression
lr = LogisticRegression(multi_class= 'ovr')
score = cross_validate(lr, X, y, return_train_score=False)
score["test_score"].mean()
lr.fit(X_TRAIN, y_train)
lr_predict = lr.predict(X_test)
# print confusion matrix and accuracy score
lr_conf_matrix = confusion_matrix(y_test, lr_predict)
lr_acc_score = accuracy_score(y_test, lr_predict)
print('Logistic Regression:')
print(lr_conf_matrix)
print(lr_acc_score*100)

# Decision Trees
dt = DecisionTreeClassifier()
dt.fit(X_TRAIN,y_train)
dt_predict = dt.predict(X_test)
#print confusion matrix and accuracy score
dt_conf_matrix = confusion_matrix(y_test, dt_predict)
dt_acc_score = accuracy_score(y_test, dt_predict)
print('Decision Trees:')
print(dt_conf_matrix)
print(dt_acc_score*100)

# NaiveBayes
nb = GaussianNB()
nb.fit(X_TRAIN,y_train)
nb_predict=nb.predict(X_test)
#print confusion matrix and accuracy score
nb_conf_matrix = confusion_matrix(y_test, nb_predict)
nb_acc_score = accuracy_score(y_test, nb_predict)
print('NaiveBayes:')
print(nb_conf_matrix)
print(nb_acc_score*100)

# RandomForest
rf = RandomForestClassifier()
rf.fit(X_TRAIN, y_train)
rf_predict=rf.predict(X_test)
#print confusion matrix and accuracy score
rf_conf_matrix = confusion_matrix(y_test, rf_predict)
rf_acc_score = accuracy_score(y_test, rf_predict)
print('RandomForest:')
print(rf_conf_matrix)
print(rf_acc_score*100)

#first test the linear kernel first and check the accuracy
lin_svc = SVC(kernel='linear')
lin_svc.fit(X_TRAIN, y_train)
lin_svc=lin_svc.predict(X_test)
#print confusion matrix and accuracy score
lin_svc_conf_matrix = confusion_matrix(y_test, rf_predict)
lin_svc_acc_score = accuracy_score(y_test, rf_predict)
print('Linear SVC:')
print(lin_svc_conf_matrix)
print(lin_svc_acc_score*100)

#second try the rbf kernel
rbf_svc = SVC(kernel='rbf')
rbf_svc.fit(X_TRAIN, y_train)
rbf_svc=rbf_svc.predict(X_test)
rbf_svc_conf_matrix = confusion_matrix(y_test, rf_predict)
rbf_svc_acc_score = accuracy_score(y_test, rf_predict)
print('rbf SVC:')
print(rbf_svc_conf_matrix)
print(rbf_svc_acc_score*100)

In [0]:
# Establish cross-validated GridSearchCV

# Model Training 
print ("Train the model ... ")
classifier = SVC(C=100, # penalty parameter, setting it to a larger value 
	 			 kernel='rbf', # kernel type, rbf working fine here
	 			 degree=3, # default value, not tuned yet
	 			 gamma=1, # kernel coefficient, not tuned yet
	 			 coef0=1, # change to 1 from default value of 0.0
	 			 shrinking=True, # using shrinking heuristics
	 			 tol=0.001, # stopping criterion tolerance 
	      		 probability=False, # no need to enable probability estimates
	      		 cache_size=200, # 200 MB cache size
	      		 class_weight=None, # all classes are treated equally 
	      		 verbose=False, # print the logs 
	      		 max_iter=-1, # no limit, let it run
          		 decision_function_shape=None, # will use one vs rest explicitly 
          		 random_state=None)
model = OneVsRestClassifier(classifier, n_jobs=4)
model.fit(X, y)


# Predictions 
print ("Predict on test data ... ")
y_test = model.predict(X_test)
y_pred = lb.inverse_transform(y_test)


# Submission
print ("Generate Submission File ... ")
test_id = [doc['id'] for doc in test]
sub = pd.DataFrame({'id': test_id, 'cuisine': y_pred}, columns=['id', 'cuisine'])
sub.to_csv('svm_output.csv', index=False)